In [227]:
# Importing libararies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt, floor
from sklearn.preprocessing import scale
from sklearn import preprocessing

#Reading files
Train_Data = pd.read_csv('trainData.csv').values
Train_Labels = pd.read_csv('trainLabels.csv').values
Test_Data = pd.read_csv('testData.csv').values
Test_Labels = pd.read_csv('testLabels.csv').values
row, col = Train_Labels.shape

m, n = Train_Labels.shape
t, _ = Test_Labels.shape
Train_Labels = Train_Labels.reshape((m))
Test_Labels = Test_Labels.reshape((t))

x_max = np.max(Train_Data)
x_min = np.min(Train_Data)

t_max = np.max(Test_Data)
t_min = np.min(Test_Data)

#train_scaled = scale(Train_Data.astype(np.float64))
#test_scaled = scale(Test_Data.astype(np.float64))
#train_normal = preprocessing.normalize(Train_Data)
#test_normal = preprocessing.normalize(Test_Data)

train_normal = (Train_Data - x_min)/(x_max - x_min)
test_normal = (Train_Data - t_min)/(t_max - t_min)

m, n = train_normal.shape
t, _ = test_normal.shape

train_normal = train_normal.reshape((m, n, 1))
test_normal = test_normal.reshape((t, n, 1))

In [229]:
train_normal.shape

(59999, 784, 1)

In [33]:
Train_Labels
L, counts = np.unique(Train_Labels, return_counts=True)
n_labels = len(L)
counts

array([6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 5999],
      dtype=int64)

In [230]:
L, counts = np.unique(Train_Labels, return_counts=True)
n_labels = len(L)

cov = np.zeros((n_labels, n, n))
means = np.zeros((n_labels, n, 1))

for i in range(m):
    for j in range(n_labels):
        if Train_Labels[i] == L[j]:
            means[j][:][:] += train_normal[i][:][:]
            
for i in range(n_labels):
    means[i][:][:] = means[i][:][:]/counts[i]



In [231]:
for i in range(m):
    for j in range(n_labels):
        if Train_Labels[i] == L[j]:
            a = train_normal[i][:][:]
            u = means[j][:][:]
            v = a-u
            o = np.multiply(v, np.transpose(v))
            cov[j][:][:] += train_normal[i][:][:] + o


In [232]:
for i in range(n_labels):
    cov[i][:][:] = cov[i][:][:]/(counts[i]-1)

In [250]:
cov_n = np.zeros((n_labels, n, n))
alpha = 0.5
for i in range(n_labels):
    sigma = (1/n)*np.trace(cov[i][:][:])
    cov_n[i][:][:] = (1-alpha)*cov[i][:][:] + alpha*sigma*np.eye(n)

In [239]:
np.linalg.det(cov_normal[7][:][:])

0.0

In [240]:
cov_inv = np.zeros((n_labels, n, n))
det = np.zeros(n_labels)

for i in range(n_labels):
    cov_inv[i][:][:] = np.linalg.inv(cov_n[i][:][:])

for i in range(n_labels):
    det[i] = np.linalg.det(cov_n[i][:][:])

In [253]:
np.trace(np.log(cov_n[8]))

-694.7042282168284

In [166]:
np.linalg.det(cov[1][:][:])

0.0

In [264]:
pred = np.zeros(t)
for i in range(t):
    temp = np.zeros(n_labels)
    for j in range(n_labels):
        a = -0.5*np.trace(np.log(cov_n[j][:][:]))
        x = test_normal[i][:][:]
        M = means[j][:][:]
        covar = cov_inv[j][:][:]
        U = np.subtract(x,M)
        UT = np.transpose(U)
        A = np.transpose(U) @ covar
        B = -0.5*(A @ U)
        temp[j] = B + a
    pred[i] = np.argmax(temp)

C:\Users\Alireza\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


In [266]:
acc = 0
for i in range(t):
    if pred[i] == Train_Labels[i]:
        acc += 1
acc = (acc/t)*100
print(f"Accuracy = {acc}")

Accuracy = 60.348839147319126


In [194]:
pred

array([0., 3., 0., ..., 3., 0., 2.])

In [181]:
a = np.array([[1,2]])
b = np.array([[3,4],
              [1,1]])
a@b

array([[5, 6]])